In [ ]:


!pip3 install -q -U accelerate==0.27.1

!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.5 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-a3885kc7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-a3885kc7
  Resolved https://github.com/huggingface/accelerate to commit e1247de01e0733c5d21075cb6f39b2605f4be123
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-

In [ ]:
!pip -qqq install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", torch_dtype=torch.float16, device_map='auto')

embedding_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embedding_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Define the training examples
training_examples = [
    {
        "sentence": "I walked 3 miles in 40 minutes. Wow, I feel excited, that was great.",
        "exercise_segment": "walked",
        "feeling_segment": "excited",
        "duration": "40 minutes",
        "intensity": "low"
    },
    {
        "sentence": "I lifted weights for 30 minutes. I felt energized after.",
        "exercise_segment": "weights",
        "feeling_segment": "energized",
        "duration": "30 minutes",
        "intensity": "high"
    },
    {
        "sentence": "I participated in running for 20 mins. I felt tired.",
        "exercise_segment": "running",
        "feeling_segment": "tired",
        "duration": "20 mins",
        "intensity": "medium"
    },
    {
        "sentence": "I did 50 push ups in 15 mins. I feel lighter.",
        "exercise_segment": "push ups",
        "feeling_segment": "lighter",
        "duration": "15 mins",
        "intensity": "low"
    },
    {
        "sentence": "I hiked 6 miles for 1 hour. I felt really sore afterwards.",
        "exercise_segment": "hiked",
        "feeling_segment": "sore",
        "duration": "1 hour",
        "intensity": "medium"
    },
    {
        "sentence": "I did HIIT training for 1 hour. I felt sore.",
        "exercise_segment": "HIIT training",
        "feeling_segment": "sore",
        "duration": "1 hour",
        "intensity": "high"
    }
]

path = '/content/testset.csv'
df_test = pd.read_csv(path)

stemmer = PorterStemmer()

def stem_word(word):
    return stemmer.stem(word)

def get_predictions(log_entry, training_examples):
    few_shot_prompt = ""
    for example in training_examples:
        few_shot_prompt += (
            f"Log: {example['sentence']}\n"
            f"Exercise Tag: {example['exercise_segment']}\n"
            f"Feeling Tag: {example['feeling_segment']}\n"
            f"Intensity: {example['intensity']}\n"
            f"Duration: {example['duration']}\n\n"
        )

    prompt = few_shot_prompt + f"Log: {log_entry}\nExercise Tag:"

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=inputs['input_ids'].shape[1] + 50)

    # Decode the prediction and extract the information
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split(prompt)[-1].strip()
    lines = prediction.split('\n')

    # Parse the predicted tags from the model output
    exercise_tag = lines[0].replace('Exercise Tag: ', '').strip()
    feeling_tag = lines[1].replace('Feeling Tag: ', '').strip()
    intensity = lines[2].replace('Intensity: ', '').strip()
    duration = lines[3].replace('Duration: ', '').strip()

    return exercise_tag, feeling_tag, intensity, duration

# Function to classify text using training examples
def classify_text(log, training_examples):
    return get_predictions(log, training_examples)

# Initialize lists to store true and predicted values
true_exercise_tags = []
true_feeling_tags = []
true_intensities = []
true_durations = []
predicted_exercise_tags = []
predicted_feeling_tags = []
predicted_intensities = []
predicted_durations = []

# Process each entry in the test set
for _, row in df_test.iterrows():
    new_log = row['sentence']
    conditioning_examples = random.sample(training_examples, min(6, len(training_examples)))
    exercise_tag, feeling_tag, intensity, duration = classify_text(new_log, conditioning_examples)

    true_exercise_tags.append(stem_word(row['exercise_segment']))
    true_feeling_tags.append(stem_word(row['feeling_segment']))
    true_intensities.append(row['intensity'])
    true_durations.append(row['duration'])

    predicted_exercise_tags.append(stem_word(exercise_tag))
    predicted_feeling_tags.append(stem_word(feeling_tag))
    predicted_intensities.append(intensity)
    predicted_durations.append(duration)

# Helper function to get embeddings
def get_embedding(text):
    inputs = embedding_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = embedding_model(**inputs).last_hidden_state.mean(dim=1).squeeze()
    return embeddings

# Calculate semantic similarity
def calculate_semantic_similarity(true_list, predicted_list):
    similarities = []
    for true_text, pred_text in zip(true_list, predicted_list):
        true_embedding = get_embedding(true_text).unsqueeze(0)
        pred_embedding = get_embedding(pred_text).unsqueeze(0)
        similarity = cosine_similarity(true_embedding.cpu().numpy(), pred_embedding.cpu().numpy())[0][0]
        similarities.append(similarity)
    return similarities

# Calculate custom accuracy
def calculate_custom_accuracy(true_tags, predicted_tags, similarities, threshold=0.7):
    correct_matches = sum(1 for true_tag, pred_tag, sim in zip(true_tags, predicted_tags, similarities) if true_tag == pred_tag or sim >= threshold)
    return correct_matches / len(true_tags)

# Calculate accuracies
exercise_tag_similarities = calculate_semantic_similarity(true_exercise_tags, predicted_exercise_tags)
feeling_tag_similarities = calculate_semantic_similarity(true_feeling_tags, predicted_feeling_tags)
intensity_tag_similarities = calculate_semantic_similarity(true_intensities, predicted_intensities)
duration_tag_similarities = calculate_semantic_similarity(true_durations, predicted_durations)

exercise_tag_accuracy = calculate_custom_accuracy(true_exercise_tags, predicted_exercise_tags, exercise_tag_similarities)
feeling_tag_accuracy = calculate_custom_accuracy(true_feeling_tags, predicted_feeling_tags, feeling_tag_similarities)
intensity_accuracy = calculate_custom_accuracy(true_intensities, predicted_intensities, intensity_tag_similarities)
duration_accuracy = calculate_custom_accuracy(true_durations, predicted_durations, duration_tag_similarities)

# Print the accuracies
print(f"Exercise Tag Accuracy: {exercise_tag_accuracy:.4f}")
print(f"Feeling Tag Accuracy: {feeling_tag_accuracy:.4f}")
print(f"Intensity Accuracy: {intensity_accuracy:.4f}")
print(f"Duration Accuracy: {duration_accuracy:.4f}")
